# Showing Live Power with Plotly

This notebook shows how PYNQ and Plotly can be used to create a live updating plot of the power used by the board. This notebook should be run in JupyterLab with the Plotly JupyterLab extensions installed.

Please refer to the official Plotly [installation instructions](https://github.com/plotly/plotly.py#installation) and the [JupyterLab support](https://github.com/plotly/plotly.py#jupyterlab-support-python-35) section for more info on how to install everything you need.

Notice that power monitoring functionalities *will not work* on Amazon AWS F1.

## Reading and recording power data

The first step is to read the power data from the board and get into a suitable format for passing to Plotly for display. Reading the data is accomplished through the `sensors` attribute of an Alveo `Device`. The `active_device` property will return the first device in the system which is what we will use for the rest of this notebook.

In [1]:
from pynq import Device

sensors = Device.active_device.sensors
sensors

{'0v85': XrtRail {name=0v85, voltage=Sensor {name=0v85, value=0.856V}},
 '12v_aux': XrtRail {name=12v_aux, voltage=Sensor {name=12v_aux_vol, value=12.133000000000001V}, current=Sensor {name=12v_aux_curr, value=1.367A}, power=Sensor {name=12v_aux_power, value=16.585811W}},
 '12v_pex': XrtRail {name=12v_pex, voltage=Sensor {name=12v_pex_vol, value=12.139000000000001V}, current=Sensor {name=12v_pex_curr, value=1.389A}, power=Sensor {name=12v_pex_power, value=16.861071000000003W}},
 '12v_sw': XrtRail {name=12v_sw, voltage=Sensor {name=12v_sw, value=12.138V}},
 '1v8': XrtRail {name=1v8, voltage=Sensor {name=1v8, value=1.834V}},
 '3v3_aux': XrtRail {name=3v3_aux, voltage=Sensor {name=3v3_aux_vol, value=3.347V}},
 '3v3_pex': XrtRail {name=3v3_pex, voltage=Sensor {name=3v3_pex_vol, value=3.362V}, current=Sensor {name=3v3_pex_curr, value=0.0A}, power=Sensor {name=3v3_pex_power, value=0.0W}},
 'mgt0v9avcc': XrtRail {name=mgt0v9avcc, voltage=Sensor {name=mgt0v9avcc, value=0.911V}},
 'mgtavtt': Xr

For measuring the power there are 3 rails of interest - the `12v_aux` and `12v_pex` rails which together account for the vast majority of power consumed by the board and the `vccint` rail which is the main FPGA power supply.

To record the power data PYNQ has a `DataRecorder` class inside the `pmbus` module which will record data from the sensors directly into a Pandas dataframe that we can use with Plotly. The constructor takes the sensors we want to record.

In [2]:
from pynq.pmbus import DataRecorder

recorder = DataRecorder(sensors["12v_aux"].power,
                        sensors["12v_pex"].power,
                        sensors["vccint"].power)

We can now get the dataframe

In [3]:
import pandas as pd

f = recorder.frame

To start recording the sensor data call `DataRecorder.record` with the sampling rate in seconds - 10 times per second in this case

In [4]:
recorder.record(0.1)

We can use Pandas to inspect the data we are recording

In [5]:
f.head()

,Mark,12v_aux_power,12v_pex_power,vccint_power
2019-12-20 17:05:58.434916,0.0,17.150994,16.781626,12.407580
2019-12-20 17:05:58.539089,0.0,16.991800,16.869752,12.366732
2019-12-20 17:05:58.641383,0.0,16.991800,16.869752,12.366732
2019-12-20 17:05:58.743488,0.0,17.663211,16.970756,12.274824
2019-12-20 17:05:58.845564,0.0,16.661355,16.740189,12.366732


## Plotting the Dataframe

First we need to create a blank graph we can populate with data. Plotly does this by having a layout dictionary in which we specify the axes and labels

In [6]:
import plotly.graph_objs as go

layout = {
    'xaxis': {
        'title': 'Time (s)'
    },
    'yaxis': {
        'title': 'Power (W)',
        'rangemode': 'tozero',
        'autorange': True
    }
}

plot = go.FigureWidget(layout=layout)
plot

FigureWidget({
    'data': [],
    'layout': {'template': '...',
               'xaxis': {'title': {'text': 'T…

Next we need a function to actually plot the data. While we could the the dataframe directly from the DataRecorder we can format it to be more immediately informative as well as only display the most recently recorded samples. Here we make use of a number of plotly functions to limit and average the data so that we get both instantaneous and moving average power over a specified period of time.

In [7]:
def update_data(frame, start, end, plot):
    ranged = frame[start:end]
    average_ranged = frame[start-pd.tseries.offsets.Second(5):end]
    rolling = (average_ranged['12v_aux_power'] + average_ranged['12v_pex_power']).rolling(
        pd.tseries.offsets.Second(5)
    ).mean()[ranged.index]
    powers = pd.DataFrame(index=ranged.index)
    powers['board_power'] = ranged['12v_aux_power'] + ranged['12v_pex_power']
    powers['rolling'] = rolling
    data = [
        go.Scatter(x=powers.index, y=powers['board_power'], name="Board Power"),
        go.Scatter(x=powers.index, y=powers['rolling'], name="5 Second Avg")
    ]
    plot.update(data=data)

To actually update the graph in a live fashion we need to create a new thread that will update our graph periodically

In [8]:
import threading
import time

do_update = True

def thread_func():
    while do_update:
        now = pd.Timestamp.fromtimestamp(time.time())
        past = now - pd.tseries.offsets.Second(60)
        update_data(recorder.frame, past, now, plot)
        time.sleep(0.5)

from threading import Thread
t = Thread(target=thread_func)
t.start()

Now the graph is updating try running other notebooks to see how power consumption changes based on load.

## Cleaning Up

To clean up we need to stop both the update thread we created and the DataRecorder

In [9]:
do_update = False
t.join()
recorder.stop()

Copyright (C) 2020 Xilinx, Inc